In [1]:
import math
from tensorboardX import SummaryWriter
import env_configurations
sess = None


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [2]:
import tensorflow as tf
import collections
gpu_options = tf.GPUOptions(allow_growth=True,per_process_gpu_memory_fraction=0.8)

config=tf.ConfigProto(gpu_options=gpu_options)
sess = tf.InteractiveSession(config=config)

In [3]:
import gym
import wrappers
import games_configurations
env_name = 'RoboschoolHumanoid-v1'#'RoboschoolHalfCheetah-v1'#'BipedalWalker-v2'#'Pendulum-v0'#'QuadruppedWalk-v1' #'QuadruppedWalk-v1' #$'BipedalWalker-v2'#'RoboschoolAnt-v1'
#'BipedalWalker-v2'#'RoboschoolAnt-v1' #'LunarLanderContinuous-v2'#'BipedalWalker-v2'#'LunarLander-v2' 
#'MountainCarContinuous-v0'#'CarRacing-v0'#'CartPole-v1' #'RoboschoolAnt-v1' #'CarRacing-v0' #'LunarLander-v2' #'Acrobot-v1' #
#a2c_config = games_configurations.pendulum_lstm_config
#a2c_config = games_configurations.halfcheetah_lstm_config_v2
#a2c_config = games_configurations.roboschoolant_config
#a2c_config = a2c_games_configurations.pendulum_config
a2c_config = games_configurations.roboschoolhumanoid_config
#a2c_config = games_configurations.roboschoolhumanoid_config
#a2c_config = games_configurations.bipedalwalker_lstm_config
obs_space, action_space = env_configurations.get_obs_and_action_spaces(env_name)
print(obs_space)
print(action_space)


Box(44,)
Box(17,)


In [4]:
import ray
ray.init(redis_max_memory=1024*1024*100, object_store_memory=1024*1024*100)

2019-07-20 12:08:32,048	INFO node.py:469 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-07-20_12-08-32_6151/logs.
2019-07-20 12:08:32,161	INFO services.py:407 -- Waiting for redis server at 127.0.0.1:46458 to respond...
2019-07-20 12:08:32,305	INFO services.py:407 -- Waiting for redis server at 127.0.0.1:63056 to respond...
2019-07-20 12:08:32,313	INFO services.py:804 -- Starting Redis shard with 0.1 GB max memory.
2019-07-20 12:08:32,341	INFO node.py:483 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-07-20_12-08-32_6151/logs.
2019-07-20 12:08:32,343	INFO services.py:1427 -- Starting the Plasma object store with 0.1 GB memory using /dev/shm.


{'node_ip_address': '192.168.3.106',
 'redis_address': '192.168.3.106:46458',
 'object_store_address': '/tmp/ray/session_2019-07-20_12-08-32_6151/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2019-07-20_12-08-32_6151/sockets/raylet',
 'webui_url': None}

In [ ]:
from a2c_continuous import A2CAgent
import tr_helpers
import networks



agent = A2CAgent(sess,'run', obs_space, False, action_space, a2c_config)
#agent.restore('nn/latest_run')
agent.train()
#agent.save('nn/latest')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.dense instead.


2019-07-20 12:08:33,624	ERROR worker.py:1672 -- WARNING: 24 workers have been started. This could be a result of using a large number of actors, or it could be a consequence of using nested tasks (see https://github.com/ray-project/ray/issues/3644) for some a discussion of workarounds.


Instructions for updating:
Use tf.cast instead.
(pid=6373) 
(pid=6373) WARNING: The TensorFlow contrib module will not be included in TensorFlow 2.0.
(pid=6373) For more information, please see:
(pid=6373)   * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
(pid=6373)   * https://github.com/tensorflow/addons
(pid=6373) If you depend on functionality not listed there, please file an issue.
(pid=6373) 
(pid=6254) 
(pid=6254) WARNING: The TensorFlow contrib module will not be included in TensorFlow 2.0.
(pid=6254) For more information, please see:
(pid=6254)   * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
(pid=6254)   * https://github.com/tensorflow/addons
(pid=6254) If you depend on functionality not listed there, please file an issue.
(pid=6254) 
(pid=6359) 
(pid=6359) WARNING: The TensorFlow contrib module will not be included in TensorFlow 2.0.
(pid=6359) For more information, please see:
(pid=6359)   * https:

Frames per seconds:  2065.7802322565503
Frames per seconds:  1894.4485490652125
saving next best rewards:  -49.8999139539424
Frames per seconds:  1929.0840816451193
saving next best rewards:  -46.74398860820598
Frames per seconds:  1836.5872980527001
Frames per seconds:  1964.0598528361504
saving next best rewards:  -45.07905078670874
Frames per seconds:  2030.0560696775376
saving next best rewards:  -44.086628330050196
Frames per seconds:  2043.9314385460802
saving next best rewards:  -41.42284763843955
Frames per seconds:  1908.8404188369984
saving next best rewards:  -37.5636287250092
Frames per seconds:  2018.857208127196
saving next best rewards:  -35.53850263235926
Frames per seconds:  2047.3646990881293
saving next best rewards:  -34.784960091089495
Frames per seconds:  1974.1443930999567
saving next best rewards:  -30.987578269729728
Frames per seconds:  2055.4023916308356
saving next best rewards:  -30.94003818499301
Frames per seconds:  2052.34464648154
saving next best rewar

In [ ]:
agent.save('nn/latest_run')
ray.shutdown()

In [ ]:
import gym
gym.envs.registry.all()